In [3]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect, realign2
from util import get_path
import pandas as pd
from datetime import datetime,timedelta
import ast
import imageio


In [4]:
plate = 13
date_init = datetime(2020,7,1,19,57)
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(4)]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [28]:
params=read_mat('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat')

In [152]:
# (23625, 14540)
xs=23625
ys=14540

In [153]:
x1=params['x1']
x2=params['x2']
y0=params['y0']
Sf=params['Sf']
L0center=params['L0center']

In [154]:
date = dates[1]

In [155]:
date_plate = f'/2020{date}'
plate_str = f'_Plate{plate}'
findingbaits = read_mat('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Analysis/FindingBaits.mat')

In [156]:
Rot= np.loadtxt('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Analysis/Skeletonrot.txt')
trans= np.loadtxt('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Analysis/Skeletontrans.txt')

In [157]:
# rottrans=np.dot(np.linalg.inv(Rot),np.array([xs,ys]))-trans
# xs,ys=round(rottrans[0]),round(rottrans[1])

In [158]:
Rcenter=findingbaits['Rcenter']
Rradii = findingbaits['Rradii']
Lcenter = findingbaits['Lcenter']
t = findingbaits['t']
Lradii = findingbaits['Lradii']

In [159]:
path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'

In [160]:
tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)

In [161]:
xs_yss=list(tileconfig[2])

In [162]:
xes= [xs_ys[0] for xs_ys in xs_yss]
yes= [xs_ys[1] for xs_ys in xs_yss]

In [163]:
cmin=np.min(xes)
cmax=np.max(xes)
rmin = np.min(yes)
rmax = np.max(yes)

In [164]:
S0     = [rmax+3000+abs(rmin),cmax+4096+abs(cmin)]
theta  = np.arctan((Rcenter[1]-Lcenter[1])/(Rcenter[0]-Lcenter[0]))
Sry    =  S0[0]*np.cos(abs(theta))+S0[1]*np.sin(abs(theta))

In [165]:
C = np.tan(-np.arctan((ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)*np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/(xs + (x2+x1)/2 - (Sf[0]-y0) - 1 - L0center[0] + np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)*np.cos(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))) - theta)
ximg = - (ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)*np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/(np.sqrt(1+C**2)*np.sin(np.arctan(C)+theta)) 
yimg = S0[0] - C*ximg;

In [166]:
def find(xsub,ysub,x,y):
    indexes=[]
    for i in range(len(xsub)):
        if x>=xsub[i]-cmin and x<xsub[i]-cmin+4096 and y>=ysub[i]-rmin and y < ysub[i]-rmin+3000:
            indexes.append(i)
    return(indexes)

In [167]:
indsImg = find(xes,yes,ximg,yimg)
possImg  = [ximg-np.array(xes)[indsImg]+cmin+1, yimg-np.array(yes)[indsImg]+rmin+1];

In [172]:
indsImg

[15, 30, 97, 112]

In [168]:
paths=[]
for index in indsImg:
    pipeline.paths.append('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+tileconfig[0][index])

In [186]:
paths

['//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/20200701_2357_Plate13/Img/Img_r02_c01.tif',
 '//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/20200701_2357_Plate13/Img/Img_r03_c01.tif',
 '//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/20200701_2357_Plate13/Img/Img_r07_c08.tif',
 '//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/20200701_2357_Plate13/Img/Img_r08_c08.tif']

In [183]:
i=3

In [184]:
im = imageio.imread(paths[i])


In [187]:
plot_t_tp1([],[],{0:(possImg[1][i],possImg[0][i])},None,im,im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …